In [1]:
import os 
import pathlib

import traci
import sumo_rl
from pettingzoo.utils.conversions import parallel_wrapper_fn
from environment.reward_functions import combined_reward

from environment.observation import Grid2x2ObservationFunction

os.environ['SUMO_HOME'] = '/opt/homebrew/opt/sumo/share/sumo'
os.environ['RAY_PICKLE_VERBOSE_DEBUG'] = '1'

## Create parallel environment API using SUMO-RL + rllib

In [2]:
import ray
ray.shutdown()
ray.init()

/Users/loveen/.pyenv/versions/3.9.10/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-27 10:55:09,464	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-01-27 10:55:11,086	INFO worker.py:1673 -- Started a local Ray instance.


Python version:,3.9.10
Ray version:,2.8.0


In [3]:
from sumo_rl.environment.env import env, parallel_env
from ray.tune import register_env
from ray.rllib.env.wrappers.pettingzoo_env import ParallelPettingZooEnv
from ray.rllib.env.wrappers.multi_agent_env_compatibility import MultiAgentEnvCompatibility

from environment.envs import RealMultiAgentSumoEnv

import random

env_folder = "data/2x2grid"

env_name = "2x2grid"

seed = 2
random.seed(seed)

metrics_csv = os.path.join("metrics", "2x2")

env_params = {"net_file": os.path.join(env_folder, "2x2.net.xml"),
        "route_file": os.path.join(env_folder, "2x2.rou.xml"),
        "reward_fn": combined_reward,
        "observation_class": Grid2x2ObservationFunction, 
        "out_csv_name": "outputs/2x2grid/ppo", 
        "num_seconds": 2000,
        "add_per_agent_info": True,
        "add_system_info": True,
        "sumo_seed": seed,
        "single_agent": False}
 
# We need to use env = parallel_env(**args) to create a petting zoo env through SUMO-RL
# With this environment, we need to override some of the methods within it, for example keep count of the evaluation metrics, etc
# We need to also make sure that we override the count methods ... 

sumo_pz_env = RealMultiAgentSumoEnv(**env_params) # SUMO environment implementing PettingZoo API

# env1 = MultiAgentSumoEnv(eval=True, csv_path=metrics_csv, **env_params)
# # env3 = MultiAgentEnvCompatibility(env1) # find out what exactly this does. doesnt work without it!

from ray.rllib.env import PettingZooEnv, ParallelPettingZooEnv

rllib_compat_pz_env = PettingZooEnv(sumo_pz_env)

register_env(name=env_name, env_creator= lambda config : rllib_compat_pz_env)
# # parallel_env() # takes a bunch of configs, instantiates a pettingzoo env

2024-01-27 10:55:11,783	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


 Retrying in 1 seconds


Error: File 'data/2x2grid/2x2.net.xml' is not accessible (No such file or directory).
Quitting (on error).


FatalTraCIError: Connection closed by SUMO.

(raylet) [2024-01-27 10:55:21,044 E 2556 35954] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-01-27_10-55-09_506310_2517 is over 95% full, available space: 3200823296; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2024-01-27 10:55:31,134 E 2556 35954] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-01-27_10-55-09_506310_2517 is over 95% full, available space: 3200757760; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2024-01-27 10:55:41,137 E 2556 35954] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-01-27_10-55-09_506310_2517 is over 95% full, available space: 3199541248; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2024-01-27 10:55:51,138 E 2556 35954] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-01-27_10-55-09_506310_2517 is over 95% full, available space: 3199442944; capacity: 494384795648. Object creation will fail i

In [ ]:
rllib_compat_pz_env.get_agent_ids()

{'1', '2', '5', '6'}

In [ ]:
rllib_compat_pz_env.env.rewards

{'1': 0, '2': 0, '5': 0, '6': 0}

In [ ]:
# print(env.action_space)
# print(env.observation_space)
# print(env.get_agent_ids())
# print(env.action_space_sample())
# print(env.get_sub_environments)
# env.get_sub_environments

## Create algorithm config

In [ ]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.logger import pretty_print

ppo_config: PPOConfig

ppo_config = (
    PPOConfig() # configuration class, initialising it returns an object self (config object)
    .environment(env_name, 
                 disable_env_checking=False)
    .framework(framework="torch")
    .rollouts(
        rollout_fragment_length=100,
        num_rollout_workers=10,
    )
    .training(
        lambda_=0.95,
        kl_coeff=0.5,
        clip_param=0.1,
        vf_clip_param=10.0,
        entropy_coeff=0.01,
        train_batch_size=1000,
        sgd_minibatch_size=100,
        num_sgd_iter=10,
    )
    .debugging(
        seed=seed, 
        log_level='DEBUG',
        )
    .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
    .multi_agent(
        policies=set(rllib_compat_pz_env.get_agent_ids()),
        policy_mapping_fn=(lambda agent_id, *args, **kwargs: agent_id),
    )
    .fault_tolerance(recreate_failed_workers=True)
)

2023-11-27 22:00:35,853	WARNING __init__.py:10 -- PG has/have been moved to `rllib_contrib` and will no longer be maintained by the RLlib team. You can still use it/them normally inside RLlib util Ray 2.8, but from Ray 2.9 on, all `rllib_contrib` algorithms will no longer be part of the core repo, and will therefore have to be installed separately with pinned dependencies for e.g. ray[rllib] and other packages! See https://github.com/ray-project/ray/tree/master/rllib_contrib#rllib-contrib for more information on the RLlib contrib effort.


In [ ]:
from datetime import datetime
from ray import tune

current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
dir_path = os.path.join("ray_results", env_name)


tune.run(
    "PPO",
    name=f"PPO_{current_time}",
    num_samples=10,
    # stop={"timesteps_total": 2e5},
    time_budget_s=200,
    checkpoint_freq=10,
    config=ppo_config.to_dict(),
    )

2023-11-27 22:00:35,863	INFO tune.py:586 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
2023-11-27 22:00:35,865	WARNING __init__.py:10 -- A3C has/have been moved to `rllib_contrib` and will no longer be maintained by the RLlib team. You can still use it/them normally inside RLlib util Ray 2.8, but from Ray 2.9 on, all `rllib_contrib` algorithms will no longer be part of the core repo, and will therefore have to be installed separately with pinned dependencies for e.g. ray[rllib] and other packages! See https://github.com/ray-project/ray/tree/master/rllib_contrib#rllib-contrib for more information on the RLlib contrib effort.
2023-11-27 22:00:35,866	WARNING __init__.py:10 -- A2C has/have been moved to `rllib_contrib` and will no longer be maintained by the RLlib team. You can still use it/them normally inside RLlib util Ray 2.8, 

(raylet) [2023-11-27 22:00:39,387 E 23964 385387] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-11-27_22-00-27_820320_23948 is over 95% full, available space: 6562693120; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2023-11-27 22:00:49,484 E 23964 385387] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-11-27_22-00-27_820320_23948 is over 95% full, available space: 6469312512; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2023-11-27 22:00:59,583 E 23964 385387] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-11-27_22-00-27_820320_23948 is over 95% full, available space: 6469242880; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2023-11-27 22:01:09,679 E 23964 385387] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-11-27_22-00-27_820320_23948 is over 95% full, available space: 6468939776; capacity: 494384795648. Object creation

In [ ]:
checkpoint_dir = algo.save().checkpoint.path
print(f"Checkpoint saved in directory {checkpoint_dir}")

NameError: name 'algo' is not defined

(raylet) [2023-11-27 22:04:00,996 E 23964 385387] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-11-27_22-00-27_820320_23948 is over 95% full, available space: 6464598016; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2023-11-27 22:04:11,098 E 23964 385387] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-11-27_22-00-27_820320_23948 is over 95% full, available space: 6464540672; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2023-11-27 22:04:21,193 E 23964 385387] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-11-27_22-00-27_820320_23948 is over 95% full, available space: 6462377984; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2023-11-27 22:04:31,293 E 23964 385387] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-11-27_22-00-27_820320_23948 is over 95% full, available space: 6462210048; capacity: 494384795648. Object creation

In [6]:
import numpy as np 

a = np.array([2,3])



In [8]:
a += [2]

In [10]:
a+=4

In [11]:
a

array([8, 9])

(raylet) [2024-01-27 12:54:25,081 E 2556 35954] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-01-27_10-55-09_506310_2517 is over 95% full, available space: 3171512320; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2024-01-27 12:54:35,088 E 2556 35954] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-01-27_10-55-09_506310_2517 is over 95% full, available space: 3171426304; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2024-01-27 12:54:45,101 E 2556 35954] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-01-27_10-55-09_506310_2517 is over 95% full, available space: 3171250176; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2024-01-27 12:54:55,199 E 2556 35954] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-01-27_10-55-09_506310_2517 is over 95% full, available space: 3170967552; capacity: 494384795648. Object creation will fail i

In [15]:
100%100

0

(raylet) [2024-01-27 18:17:09,226 E 2556 35954] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-01-27_10-55-09_506310_2517 is over 95% full, available space: 3106869248; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2024-01-27 18:17:19,232 E 2556 35954] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-01-27_10-55-09_506310_2517 is over 95% full, available space: 3106791424; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2024-01-27 18:17:29,332 E 2556 35954] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-01-27_10-55-09_506310_2517 is over 95% full, available space: 3107446784; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2024-01-27 18:17:39,344 E 2556 35954] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-01-27_10-55-09_506310_2517 is over 95% full, available space: 3107553280; capacity: 494384795648. Object creation will fail i

In [16]:
1-True

0

(raylet) [2024-01-27 19:01:14,441 E 2556 35954] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-01-27_10-55-09_506310_2517 is over 95% full, available space: 3085135872; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2024-01-27 19:01:24,453 E 2556 35954] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-01-27_10-55-09_506310_2517 is over 95% full, available space: 3084939264; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2024-01-27 19:01:34,462 E 2556 35954] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-01-27_10-55-09_506310_2517 is over 95% full, available space: 3084853248; capacity: 494384795648. Object creation will fail if spilling is required.
(raylet) [2024-01-27 19:01:44,471 E 2556 35954] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-01-27_10-55-09_506310_2517 is over 95% full, available space: 3084746752; capacity: 494384795648. Object creation will fail i